In [3]:
from utils import set_root 

set_root()

Current working directory: /home/guido/github/physioex-private


In [4]:
from physioex.data import PhysioExDataset
import os
import numpy as np

In [14]:

dataset_name = "mass"
prep = "raw"
data_folder = "/mnt/guido-data/"
selected_channels = ["EEG", "EOG", "EMG"]
dataset = PhysioExDataset(
    datasets = [ dataset_name ],
    data_folder= data_folder,
    preprocessing = prep,
    sequence_length=1,
    selected_channels= selected_channels
)

X = []

for i in range(len(dataset)):
    x, _ = dataset.__getitem__(i, scale=False)
    X.append( x.numpy() )

X = np.array(X)
X = X.reshape(-1, len(selected_channels), 3000 )

mean = np.mean(X, axis=0)
std = np.std(X, axis=0)

saved_mean, saved_std = dataset.scaling[0]

#print( "Mean difference: ", np.abs(mean - saved_mean.numpy()).mean() )
#print( "Std difference: ", np.abs(std - saved_std.numpy()).mean() )

#mean = np.reshape(mean, (1, -1))
#std = np.reshape(std, (1, -1))

scaling_path = f"{data_folder}{dataset_name}/{prep}/scaling.npz"
np.savez(scaling_path, mean=mean, std=std)

2024-08-25 11:25:19.596 | INFO     | physioex.utils.data_folder:set_data_folder:25 - Data folder set to /mnt/guido-data/


IndexError: index 1 is out of bounds for axis 0 with size 1

: 

In [13]:
# check if the data is now scaled
dataset = None
dataset = PhysioExDataset(
    datasets = [ dataset_name ],
    data_folder= data_folder,
    preprocessing = prep,
    sequence_length=1
)

X = []

for i in range(len(dataset)):
    x, _ = dataset.__getitem__(i)
    X.append( x.numpy() )

X = np.array(X)
X = X.reshape(-1, X.shape[-1])

print( "Mean :", np.mean(X, axis=0).mean() )
print( "Std :", np.std(X, axis=0).mean() )

saved_mean, saved_std = dataset.scaling[0]

print( "Mean difference: ", np.abs(mean - saved_mean.numpy()).mean() )
print( "Std difference: ", np.abs(std - saved_std.numpy()).mean() )

2024-08-25 11:11:56.529 | INFO     | physioex.utils.data_folder:set_data_folder:25 - Data folder set to /mnt/guido-data/


Mean : 4.844009e-10
Std : 1.0000068
Mean difference:  0.0
Std difference:  0.0


In [7]:
print(scaling_path)


/mnt/guido-data/hmc/raw/scaling.npz


In [9]:
print( dataset.scaling[0] )

(tensor([-0.0093]), tensor([32.6995]))
